In [ ]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib
import numba
import time
from numba import cuda
from numpy.random import rand as ra

In [ ]:
#function to run in GPU
@cuda.jit()
def tempfunction(xc,yc,Tp,T,Ts):
  tx = cuda.threadIdx.x
  bx = cuda.blockIdx.x
  bw = cuda.blockDim.x
  index = tx + bx * bw
  err=1
  while err>0.000001:
    mx=0
    for i in range(50):
      for j in range(50):
        vtemp=0
        for x in range(2):
          if xc[index,x]==i and yc[index,x]==j:
            vtemp=x+1
            break
        if vtemp>0:
          Ts[index,i,j]=Tp[index,vtemp-1]
        elif (int(i==0) or int(j==0) or int(i==49) or int(j==49))>0:
          Ts[index,i,j]=0
        else :
          Ts[index,i,j]=(Ts[index,i+1,j]+Ts[index,i-1,j]+Ts[index,i,j+1]+Ts[index,i,j-1])/4
    
        vt=T[index,i,j]-Ts[index,i,j]
        if vt<0:
          vt=-vt
        if vt>mx:
          mx=vt
        T[index,i,j]=Ts[index,i,j]
    err=mx


In [ ]:
#parameters 
xc=[]
yc=[]
Tc=[]
T=[]
Tt=[[0.0] * 50] * 50
for xc1 in np.arange(0,1.1,0.2):
  for xc2 in np.arange(0,1.1,0.2):
    for yc1 in np.arange(0,1.1,0.2):
      for yc2 in np.arange(0,1.1,0.2):
        for Tc1 in np.arange(0,501,100):
          for Tc2 in np.arange(0,501,100):
            xc.append([ra(),ra()])
            yc.append([ra(),ra()])
            Tc.append([ra()*500,ra()*500])
            T.append(Tt.copy())
T2=T.copy()
T=np.array(T)
T2=np.array(T2)
xc=np.array(xc)
yc=np.array(yc)
Tc=np.array(Tc)
xc=(xc*49).astype(int)
yc=(yc*49).astype(int)
Tc=Tc.astype(float)

In [ ]:
blockdim = 432
griddim = int(6**6/432)
start = time.time()
T=T.astype(float)
T2=T2.astype(float)
tempfunction[griddim, blockdim](xc,yc,Tc,T,T2)
end = time.time()

In [ ]:
colums=[]
itter=0
colums.append('x0')
colums.append('x1')
colums.append('y0')
colums.append('y1')
colums.append('Tp0')
colums.append('Tp1')
for temp in range(2500):
  colums.append('T'+str(temp))
T=T.reshape(46656,2500)
df=np.concatenate((xc,yc),axis=1)
df=np.concatenate((df,Tc),axis=1)
df=np.concatenate((df,T),axis=1)
df=pd.DataFrame(df,columns=colums)

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('datar2p.csv')
!cp datar2p.csv "drive/My Drive/"